In [1]:
import numpy as np
import pandas as pd 
import os
from pandas.tseries.holiday import USFederalHolidayCalendar
from glob import glob
import matplotlib.pyplot as plt

import utils
# from utils import load_data, get_train_val_split, get_stratified_splitter
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedGroupKFold, train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer

from lightgbm import LGBMRegressor
from scipy.stats import kstest, kruskal, mannwhitneyu
from itertools import combinations
from collections import defaultdict
from tqdm import tqdm
from sklearn.ensemble import StackingRegressor
import re
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_log_error, mean_squared_error, silhouette_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

import optuna
import pickle

In [2]:
import importlib
importlib.reload(utils)

<module 'utils' from '/Users/sharaddargan/Documents/Course Content/Spring 2023/Probability and Statistics 2/Project/EnergyPrediction-ASHRAE/code/utils.py'>

In [3]:
data_dict = utils.load_data('ashrae-energy-prediction')

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.88%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 3.07 MB
Decreased by 68.05%
Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 5.13 MB
Decreased by 73.04%
Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 289.19 MB
Decreased by 53.12%
Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.53 MB
Decreased by 71.82%


In [4]:
# Add weather features 
weather_features = ['cloud_coverage', 'dew_temperature', 'air_temperature', 
                    'sea_level_pressure', 'wind_direction', 'wind_speed', 'precip_depth_1_hr',]

hourly_by_site = data_dict["X_train"].groupby(['hour', 'month', 'site_id'])[weather_features].mean().reset_index()

data_dict["X_train"] = data_dict["X_train"].merge(
    hourly_by_site, 
    on=['hour', 'month', 'site_id'], 
    how='left', 
    suffixes=(None, '_hourly_by_site')
)

del hourly_by_site

for feature in weather_features:
    # Fill in NA values from weather with hourly by site columns 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature + "_hourly_by_site"],
        inplace=True
    )
    
    # Fill in the rest with the median 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(),
        inplace=True
    )
    
    data_dict["X_train"][feature + "_diff_hourly_from_mean"] = data_dict["X_train"][feature] - \
        data_dict["X_train"][feature + "_hourly_by_site"]
    
data_dict["X_train"] = data_dict["X_train"].drop(columns = [feat + "_hourly_by_site" for feat in weather_features])

In [5]:
# Fill in NA with median values for floor count and year_built
for feature in ['year_built', 'floor_count']:
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(), 
        inplace=True
    )

In [6]:
data_dict.keys()

dict_keys(['weather_test', 'X_train', 'X_test', 'y_train'])

## Examine Differences (Non-Parametric)
Using Bonferonni's Correction

### Milestone 2. 
- Show difference in sites across meter readings 
- get average meter reading per day per site 
- conduct a (non-parametric ANOVA) KS OR pairs (mann-whitney) to show that they are diff 
- train a model per site id (with rudimentary hyperparameter tuning) 
- John sites 0-7, Sharad sites 8-15 

### Milestone 3. Determine, per site, which primary uses are similar (if they have only a few buildings), which are diff
- for a given primary use, if diff, identify "clusters" of buildings that are similar 

In [7]:
features = ['year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed',
       'air_temperature_mean_lag7', 'air_temperature_max_lag7',
       'air_temperature_min_lag7', 'air_temperature_std_lag7',
       'cloud_coverage_mean_lag7', 'cloud_coverage_max_lag7',
       'cloud_coverage_min_lag7', 'cloud_coverage_std_lag7',
       'dew_temperature_mean_lag7', 'dew_temperature_max_lag7',
       'dew_temperature_min_lag7', 'dew_temperature_std_lag7',
       'precip_depth_1_hr_mean_lag7', 'precip_depth_1_hr_max_lag7',
       'precip_depth_1_hr_min_lag7', 'precip_depth_1_hr_std_lag7',
       'sea_level_pressure_mean_lag7', 'sea_level_pressure_max_lag7',
       'sea_level_pressure_min_lag7', 'sea_level_pressure_std_lag7',
       'wind_direction_mean_lag7', 'wind_direction_max_lag7',
       'wind_direction_min_lag7', 'wind_direction_std_lag7',
       'wind_speed_mean_lag7', 'wind_speed_max_lag7', 'wind_speed_min_lag7',
       'wind_speed_std_lag7', 'log_square_feet', 'weekday', 'hour', 'day',
       'weekend', 'month', 'primary_use_enc']

In [8]:
X_full, y_full = data_dict['X_train'].loc[:, features+['meter', 'site_id', 'building_id', 'timestamp']], data_dict['y_train']


In [9]:
building_cluster_mapping = pd.DataFrame()

silhouette_scores = {}
for site in X_full['site_id'].unique():
    for meter in X_full['meter'].unique():
        X = X_full[(X_full["site_id"]==site)&(X_full["meter"]==meter)]
        y = y_full[(X_full["site_id"]==site)&(X_full["meter"]==meter)]
        X["meter_reading"] = y

        X["YMD"] = X["timestamp"].dt.strftime("%Y-%m-%d")
        day_level_df = X.groupby(["building_id", "YMD"]).agg(meter_reading = ("meter_reading", "mean")).reset_index()
        day_level_pivot = pd.pivot_table(day_level_df, values="meter_reading", columns="YMD", index="building_id").fillna(0)
        if X.shape[0]==0:
            continue
        
        
        key = "_".join([str(site), str(meter)])
        
        silhouette_scores[key] = []
        n_buildings = day_level_pivot.shape[0]
        n_pc_range = range(2, min(n_buildings - 1, 10))
        k_range = range(2, min(5, n_buildings - 1))
        #Default best_k, best_pc values
        best_k, best_pc, best_score, best_labels = 1, 2, 0, np.ones(n_buildings)
        for n_pc in n_pc_range:
            pca = PCA(n_pc)
            t = pca.fit_transform(day_level_pivot)
            for k in k_range:
                kmeans_labels = KMeans(n_clusters=k, random_state=0).fit(t[:, :n_pc]).labels_
                ss = silhouette_score(t[:, :n_pc], kmeans_labels)
                if ss > best_score:
                    best_pc, best_k, best_score = n_pc, k, ss
                    best_labels = kmeans_labels
        
        print(site, meter, best_pc, best_k)
        
        day_level_pivot["cluster"] = best_labels
        day_level_pivot = day_level_pivot.reset_index()
        bc_mapping = day_level_pivot[["building_id", "cluster"]]
        bc_mapping["meter"] = meter
        building_cluster_mapping = pd.concat([building_cluster_mapping, bc_mapping])

1 0 2 2
1 3 2 2
2 0 2 2
2 3 2 2
2 1 2 2
3 0 2 4
4 0 2 2
5 0 2 2
6 0 2 4
6 1 2 3
6 2 2 4
7 0 2 2
7 3 2 1
7 1 2 2
7 2 2 3
8 0 2 2
9 0 2 2
9 1 2 3
9 2 2 4
10 0 2 2
10 3 2 2
10 1 2 2
11 0 2 3
11 3 2 2
11 1 2 2
12 0 2 3
13 0 2 2
13 1 2 2
13 2 2 2
14 0 2 4
14 3 2 2
14 1 2 2
14 2 2 2
15 0 2 2
15 3 2 1
15 1 2 2
15 2 2 4
0 0 2 3
0 1 2 3


In [10]:
X_full = X_full.reset_index().merge(building_cluster_mapping, on=["building_id", "meter"], how="left").set_index('index')

In [11]:
splitter_gen = utils.get_stratified_splitter(X_full, y_full)
train_index, test_index = next(splitter_gen)
X_train, y_train, X_test, y_test = X_full.loc[train_index, :], y_full[train_index], X_full.loc[test_index, :], y_full[test_index]

In [12]:
X_train.columns

Index(['year_built', 'floor_count', 'air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed', 'air_temperature_mean_lag7',
       'air_temperature_max_lag7', 'air_temperature_min_lag7',
       'air_temperature_std_lag7', 'cloud_coverage_mean_lag7',
       'cloud_coverage_max_lag7', 'cloud_coverage_min_lag7',
       'cloud_coverage_std_lag7', 'dew_temperature_mean_lag7',
       'dew_temperature_max_lag7', 'dew_temperature_min_lag7',
       'dew_temperature_std_lag7', 'precip_depth_1_hr_mean_lag7',
       'precip_depth_1_hr_max_lag7', 'precip_depth_1_hr_min_lag7',
       'precip_depth_1_hr_std_lag7', 'sea_level_pressure_mean_lag7',
       'sea_level_pressure_max_lag7', 'sea_level_pressure_min_lag7',
       'sea_level_pressure_std_lag7', 'wind_direction_mean_lag7',
       'wind_direction_max_lag7', 'wind_direction_min_lag7',
       'wind_direction_std_lag7', 'wind_speed_mean_lag7',
       'wind_speed_ma

In [11]:
X_train_w_cluster = X_train.reset_index().merge(building_cluster_mapping, on=["building_id", "meter"], how="left").set_index('index')

In [12]:
X_train_w_cluster.shape

(14892330, 49)

In [13]:
SITE_FILTER = X_train_w_cluster["site_id"] == 1
METER_FILTER = X_train_w_cluster["meter"] == 1

SITE_FILTER&METER_FILTER

index
0           False
2           False
3           False
7           False
8           False
            ...  
19852312    False
19852313    False
19852314    False
19852315    False
19852319    False
Length: 14892330, dtype: bool

In [13]:
def run_optuna_search_cv(
    site: int,
    meter: int,
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
    features: list,
    n_trials: int = 50,
    
):
    """
    Runs Optuna Search for LGBMRegressor
    """
    SITE_FILTER = X_train["site_id"] == site
    METER_FILTER = X_train["meter"] == meter
    
    X, y = X_train.loc[SITE_FILTER&METER_FILTER, features], y_train[SITE_FILTER&METER_FILTER]
    if X.shape[0] == 0:
        return None

    splitter_gen = utils.get_stratified_splitter(X_train[SITE_FILTER&METER_FILTER], y_train[SITE_FILTER&METER_FILTER])

    

    regressor = LGBMRegressor()

    param_distributions = {
        "max_depth": optuna.distributions.IntDistribution(-1, len(X_train.columns)),
        "num_leaves": optuna.distributions.IntDistribution(5, 50),
        "learning_rate": optuna.distributions.FloatDistribution(1e-7, 1, log=True),
        "n_estimators": optuna.distributions.IntDistribution(1, 500),
        "reg_alpha": optuna.distributions.FloatDistribution(1e-7, 1e7, log=True),
        "reg_lambda": optuna.distributions.FloatDistribution(1e-7, 1e7, log=True),            
    }

    """
    Parameters not searched over: 
    subsample_for_bin: int = 200000,
    min_split_gain: float = 0.0,
    min_child_weight: float = 0.001,
    min_child_samples: int = 20,
    subsample: float = 1.0,
    subsample_freq: int = 0,
    colsample_bytree: float = 1.0,
    random_state: Union[int, numpy.random.mtrand.RandomState, NoneType] = None,
    n_jobs: int = -1,
    """
    def rmse(estimator, X_test, y_test):
        y_pred = estimator.predict(X_test)
        return -1 * mean_squared_error(y_test, y_pred, squared=False)
    
    
    optuna_search = optuna.integration.OptunaSearchCV(
        regressor, 
        param_distributions,
        n_trials=n_trials,
        cv = splitter_gen,
        random_state=0, # IMPORTANT,
        refit=True,
        n_jobs=4,
        scoring = rmse,
        verbose=optuna.logging.ERROR
    )

    optuna_search.fit(X, y)
    y_pred = optuna_search.predict(X)
    
    return optuna_search

In [14]:
# This fits sites 0-7
N_TRIALS = 50
models = defaultdict(dict)

for site in tqdm(X_train["site_id"].unique()):
    for meter in X_train["meter"].unique():
        clusters = X_train.loc[(X_train["site_id"]==site)&(X_train["meter"]==meter), "cluster"].unique()
        for cluster in clusters:
            optuna_search = run_optuna_search_cv(site, \
                                                 meter, \
                                                 X_train[X_train["cluster"] == cluster], \
                                                 y_train[X_train["cluster"] == cluster], \
                                                 features, \
                                                 N_TRIALS)

            model_identifier = "_".join([str(site), str(meter), str(cluster)])
            models[model_identifier] = optuna_search

  0%|                                                                             | 0/16 [00:00<?, ?it/s][I 2023-04-28 22:29:24,333] A new study created in memory with name: no-name-e6256404-4676-43ef-8f3d-b2768ecdd75a
[I 2023-04-28 22:29:24,335] Searching the best hyperparameters using 193248 samples...
[I 2023-04-28 22:29:29,996] Trial 3 finished with value: -0.8688835279319695 and parameters: {'max_depth': 40, 'num_leaves': 23, 'learning_rate': 0.49461950355275924, 'n_estimators': 397, 'reg_alpha': 903033.6458276437, 'reg_lambda': 2.148135041241092}. Best is trial 3 with value: -0.8688835279319695.
[I 2023-04-28 22:29:30,896] Trial 0 finished with value: -0.8688815106993231 and parameters: {'max_depth': 1, 'num_leaves': 40, 'learning_rate': 0.00010512414216169257, 'n_estimators': 382, 'reg_alpha': 4.7489426464339735, 'reg_lambda': 4988563.133357003}. Best is trial 0 with value: -0.8688815106993231.
[I 2023-04-28 22:29:36,689] Trial 4 finished with value: -0.8688835279319695 and para

[I 2023-04-28 22:30:54,426] Trial 26 finished with value: -0.86889134019371 and parameters: {'max_depth': 3, 'num_leaves': 27, 'learning_rate': 1.2987404232993388e-06, 'n_estimators': 450, 'reg_alpha': 10064.97904879817, 'reg_lambda': 0.17517177316479676}. Best is trial 0 with value: -0.8688815106993231.
[I 2023-04-28 22:30:57,438] Trial 29 finished with value: -0.8688835279319695 and parameters: {'max_depth': 5, 'num_leaves': 25, 'learning_rate': 2.1381489223335542e-07, 'n_estimators': 211, 'reg_alpha': 1187968.7162582958, 'reg_lambda': 4239.603183183222}. Best is trial 0 with value: -0.8688815106993231.
[I 2023-04-28 22:30:57,601] Trial 27 finished with value: -0.8689102811015675 and parameters: {'max_depth': 2, 'num_leaves': 27, 'learning_rate': 1.9231806931138436e-06, 'n_estimators': 442, 'reg_alpha': 13106.465851666751, 'reg_lambda': 0.575134465715117}. Best is trial 0 with value: -0.8688815106993231.
[I 2023-04-28 22:30:58,068] Trial 28 finished with value: -0.8688835279319695 an

[I 2023-04-28 22:31:49,680] A new study created in memory with name: no-name-6e48487f-2d5d-4fa8-9af7-7c13d0e54cea
[I 2023-04-28 22:31:49,682] Searching the best hyperparameters using 140542 samples...
[I 2023-04-28 22:31:55,378] Trial 3 finished with value: -1.0871855188517379 and parameters: {'max_depth': 47, 'num_leaves': 12, 'learning_rate': 0.09002092045670632, 'n_estimators': 125, 'reg_alpha': 1.0330011659569863e-05, 'reg_lambda': 4849.190256114186}. Best is trial 3 with value: -1.0871855188517379.
[I 2023-04-28 22:32:04,990] Trial 1 finished with value: -0.8483547128019471 and parameters: {'max_depth': 25, 'num_leaves': 22, 'learning_rate': 7.502983358161869e-06, 'n_estimators': 271, 'reg_alpha': 2.5021079099490948e-05, 'reg_lambda': 258670.5634937642}. Best is trial 1 with value: -0.8483547128019471.
[I 2023-04-28 22:32:12,981] Trial 2 finished with value: -0.870673500818193 and parameters: {'max_depth': 20, 'num_leaves': 14, 'learning_rate': 0.0006688314249320875, 'n_estimators

[I 2023-04-28 22:32:55,363] Trial 28 finished with value: -0.8929098909327724 and parameters: {'max_depth': 38, 'num_leaves': 33, 'learning_rate': 0.15326654104277604, 'n_estimators': 3, 'reg_alpha': 3500.748947127319, 'reg_lambda': 0.6126062441621103}. Best is trial 10 with value: -0.8483422917565611.
[I 2023-04-28 22:32:56,466] Trial 6 finished with value: -0.8483432995018688 and parameters: {'max_depth': 49, 'num_leaves': 48, 'learning_rate': 1.3943632521369105e-07, 'n_estimators': 303, 'reg_alpha': 0.061394869288020675, 'reg_lambda': 0.012830190605348337}. Best is trial 10 with value: -0.8483422917565611.
[I 2023-04-28 22:32:57,862] Trial 29 finished with value: -0.8483422917565611 and parameters: {'max_depth': 4, 'num_leaves': 50, 'learning_rate': 0.025055261116315293, 'n_estimators': 246, 'reg_alpha': 97432.95575384272, 'reg_lambda': 0.005105497333856807}. Best is trial 10 with value: -0.8483422917565611.
[I 2023-04-28 22:32:58,866] Trial 30 finished with value: -0.84834229175656

[I 2023-04-28 22:33:17,605] A new study created in memory with name: no-name-4ec73047-64af-4cc6-8acf-5eaa41b621ec
[I 2023-04-28 22:33:17,606] Searching the best hyperparameters using 43905 samples...
[I 2023-04-28 22:33:18,605] Trial 3 finished with value: -1.8732089573001274 and parameters: {'max_depth': 1, 'num_leaves': 11, 'learning_rate': 0.0005369781255170851, 'n_estimators': 119, 'reg_alpha': 0.001172999839311505, 'reg_lambda': 2713781.1654689177}. Best is trial 3 with value: -1.8732089573001274.
[I 2023-04-28 22:33:19,818] Trial 4 finished with value: -1.8733719491878884 and parameters: {'max_depth': 8, 'num_leaves': 21, 'learning_rate': 5.202706144649269e-07, 'n_estimators': 125, 'reg_alpha': 52842.98108698595, 'reg_lambda': 0.005271731228771284}. Best is trial 3 with value: -1.8732089573001274.
[I 2023-04-28 22:33:22,674] Trial 5 finished with value: -1.563480903935297 and parameters: {'max_depth': 3, 'num_leaves': 27, 'learning_rate': 0.30017619875861784, 'n_estimators': 172,

...

[I 2023-04-29 04:59:04,136] Finished refitting! (elapsed time: 0.276 sec.)
[I 2023-04-29 04:59:07,128] A new study created in memory with name: no-name-b74529b4-4c53-41eb-8a53-c2b96f9fbc47
[I 2023-04-29 04:59:07,130] Searching the best hyperparameters using 34203 samples...
[I 2023-04-29 05:16:05,055] Trial 1 finished with value: -2.6836080714714976 and parameters: {'max_depth': 39, 'num_leaves': 41, 'learning_rate': 0.0004576230115632364, 'n_estimators': 287, 'reg_alpha': 0.0005326123726068747, 'reg_lambda': 7065028.154380066}. Best is trial 1 with value: -2.6836080714714976.
[I 2023-04-29 05:16:14,177] Trial 2 finished with value: -2.683314530829774 and parameters: {'max_depth': 35, 'num_leaves': 35, 'learning_rate': 5.297553611381285e-06, 'n_estimators': 344, 'reg_alpha': 0.3162740321691827, 'reg_lambda': 1.7361964199328412e-06}. Best is trial 2 with value: -2.683314530829774.
[I 2023-04-29 05:16:15,973] Trial 4 finished with value: -3.324227629195753 and parameters: {'max_depth': 9

[I 2023-04-29 05:41:14,008] Trial 26 finished with value: -2.6789729579805135 and parameters: {'max_depth': 18, 'num_leaves': 5, 'learning_rate': 0.006795291455502396, 'n_estimators': 140, 'reg_alpha': 0.04354267841983585, 'reg_lambda': 100979.08796758713}. Best is trial 23 with value: -2.671269970683768.
[I 2023-04-29 05:41:14,944] Trial 28 finished with value: -2.8383907898236647 and parameters: {'max_depth': 15, 'num_leaves': 5, 'learning_rate': 0.008315343756630676, 'n_estimators': 127, 'reg_alpha': 1.2854986090076868, 'reg_lambda': 73.31165797292093}. Best is trial 23 with value: -2.671269970683768.
[I 2023-04-29 05:41:16,075] Trial 27 finished with value: -2.6999538462741586 and parameters: {'max_depth': 24, 'num_leaves': 5, 'learning_rate': 0.010525952743803224, 'n_estimators': 266, 'reg_alpha': 1.4168632203173566, 'reg_lambda': 152175.07813579036}. Best is trial 23 with value: -2.671269970683768.
[I 2023-04-29 05:41:16,226] Trial 24 finished with value: -2.683548524948841 and p

[I 2023-04-29 05:57:50,287] A new study created in memory with name: no-name-2b4ca879-06b6-467b-b3d0-efe9f9bee56b
[I 2023-04-29 05:57:50,289] Searching the best hyperparameters using 11486 samples...
[I 2023-04-29 05:57:52,008] Trial 3 finished with value: -2.5730632302762615 and parameters: {'max_depth': 38, 'num_leaves': 33, 'learning_rate': 2.8437639856932786e-06, 'n_estimators': 347, 'reg_alpha': 24422.257686177516, 'reg_lambda': 31972.761931719207}. Best is trial 3 with value: -2.5730632302762615.
[I 2023-04-29 05:57:52,876] Trial 1 finished with value: -2.5729519373078316 and parameters: {'max_depth': 40, 'num_leaves': 27, 'learning_rate': 2.692079078757706e-05, 'n_estimators': 52, 'reg_alpha': 3.938056481066517, 'reg_lambda': 1355.7137098310097}. Best is trial 1 with value: -2.5729519373078316.
[I 2023-04-29 05:57:55,696] Trial 4 finished with value: -2.5815062948109597 and parameters: {'max_depth': 41, 'num_leaves': 14, 'learning_rate': 0.06332738319502874, 'n_estimators': 136,

[I 2023-04-29 06:31:33,319] Trial 24 finished with value: -2.563531803102009 and parameters: {'max_depth': 7, 'num_leaves': 41, 'learning_rate': 0.0001751158020214686, 'n_estimators': 301, 'reg_alpha': 0.0008419175790888326, 'reg_lambda': 0.0268990647742388}. Best is trial 15 with value: -2.560317914571006.
[I 2023-04-29 06:31:38,210] Trial 27 finished with value: -2.567474402578286 and parameters: {'max_depth': 10, 'num_leaves': 50, 'learning_rate': 0.0001643880886362383, 'n_estimators': 198, 'reg_alpha': 0.002386256748058087, 'reg_lambda': 186.778870388386}. Best is trial 15 with value: -2.560317914571006.
[I 2023-04-29 06:31:40,611] Trial 25 finished with value: -2.565096688114127 and parameters: {'max_depth': 0, 'num_leaves': 50, 'learning_rate': 0.00018895690422624685, 'n_estimators': 196, 'reg_alpha': 0.0026294526030585627, 'reg_lambda': 0.45896315522756387}. Best is trial 15 with value: -2.560317914571006.
[I 2023-04-29 06:31:44,925] Trial 29 finished with value: -2.605216061550

100%|████████████████████████████████████████████████████████████████| 16/16 [8:20:02<00:00, 1875.14s/it]


###### Dump the models

In [15]:
for key in models.keys():
    if models[key]:
        with open(f"./models/v4/{key}.pkl", "wb") as f:
            pickle.dump(models[key].best_estimator_, f)

In [16]:
len(models.keys())

93

In [17]:
model_files = sorted(glob('models/v4/*.pkl'))
estimators = []
for model_file in model_files:
    estimators.append((re.search(r"/(\d+_\d+_\d+)\.0\.pkl$", model_file).group(1) ,pickle.load(open(model_file,'rb'))))

In [18]:
estimators_dict = dict(estimators)

In [26]:
def predict_using_individual_estimators(X_test, y_test, estimators_dict):
    results_df = pd.DataFrame(columns=['y_true', 'y_pred'])
    for site_meter_cluster in estimators_dict.keys():

        site_id, meter, cluster = list(map(lambda x:int(x) ,site_meter_cluster.split("_")))
        # Filter input data for the specific site_id and meter
        site_meter_filter = (X_test['site_id'] == site_id) & (X_test['meter'] == meter) & (X_test['cluster'] == cluster)

        X_test_subset = X_test[site_meter_filter].drop(['meter', 'site_id', 'building_id', 'timestamp', 'cluster'], axis=1)

        # Get the true values for the subset of input data
        y_true = y_test[site_meter_filter]
        
#         print(X_test_subset.shape)
#         print(y_true.shape)
        if(X_test_subset.shape[0] == 0):
            continue

        # Make predictions for the subset of input data
        y_pred = estimators_dict[site_meter_cluster].predict(X_test_subset)

        # Add true values and predictions to results_df
        results_df = pd.concat([results_df, pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})], ignore_index=True)

        # Compute RMSE for the predictions and print the results
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        print(f"Site ID: {site_id}, Meter: {meter}, RMSE: {rmse}")
    overall_rmse = mean_squared_error(results_df["y_true"], results_df["y_pred"], squared=False)
    print(f"Overall RMSE: {overall_rmse}")
    return results_df

In [65]:
X_test_w_cluster = X_test.reset_index().merge(building_cluster_mapping, on=["building_id", "meter"], how="left").set_index('index')


In [20]:
site_id, meter, cluster = list(map(lambda x:int(x), list(estimators_dict.keys())[0].split("_")))

In [27]:
results_df = predict_using_individual_estimators(X_test, y_test, estimators_dict)

Site ID: 0, Meter: 0, RMSE: 0.9638885501881904
Site ID: 0, Meter: 0, RMSE: 0.6659047414954697
Site ID: 0, Meter: 0, RMSE: 0.811408360625821
Site ID: 0, Meter: 1, RMSE: 0.4662140463122371
Site ID: 0, Meter: 1, RMSE: 1.8928775577515013
Site ID: 0, Meter: 1, RMSE: 3.025064833878893
Site ID: 10, Meter: 0, RMSE: 0.46109759959065055
Site ID: 10, Meter: 0, RMSE: 0.3986219818465133
Site ID: 10, Meter: 1, RMSE: 1.9203875618615351
Site ID: 10, Meter: 3, RMSE: 2.5234634137243965
Site ID: 11, Meter: 0, RMSE: 0.4053266227851398
Site ID: 11, Meter: 1, RMSE: 1.2207719673578301
Site ID: 12, Meter: 0, RMSE: 0.5305472720739743
Site ID: 12, Meter: 0, RMSE: 1.031896793947483
Site ID: 13, Meter: 0, RMSE: 1.3170571112954468
Site ID: 13, Meter: 0, RMSE: 0.6780805804071306
Site ID: 13, Meter: 1, RMSE: 1.3693572342887494
Site ID: 13, Meter: 1, RMSE: 1.5574757407196498
Site ID: 13, Meter: 2, RMSE: 2.6190908628983336
Site ID: 13, Meter: 2, RMSE: 1.8454906509726696
Site ID: 14, Meter: 0, RMSE: 1.12567490156283
Si

In [29]:
results_df = predict_using_individual_estimators(X_train, y_train, estimators_dict)

Site ID: 0, Meter: 0, RMSE: 0.44104433683150934
Site ID: 0, Meter: 0, RMSE: 0.25765967966578684
Site ID: 0, Meter: 0, RMSE: 0.33405107413373625
Site ID: 0, Meter: 1, RMSE: 2.59655361597847
Site ID: 0, Meter: 1, RMSE: 1.188140022767329
Site ID: 0, Meter: 1, RMSE: 2.394168889915262
Site ID: 10, Meter: 0, RMSE: 0.9858562300777253
Site ID: 10, Meter: 0, RMSE: 0.5639514906587215
Site ID: 10, Meter: 1, RMSE: 0.030886271273774323
Site ID: 10, Meter: 1, RMSE: 1.4466369881054661
Site ID: 10, Meter: 3, RMSE: 1.7277863844689032
Site ID: 10, Meter: 3, RMSE: 3.3460710714770103
Site ID: 11, Meter: 0, RMSE: 0.048857680003833164
Site ID: 11, Meter: 1, RMSE: 0.19942568995176685
Site ID: 11, Meter: 1, RMSE: 1.0245776451305504
Site ID: 11, Meter: 3, RMSE: 0.7873662999346656
Site ID: 11, Meter: 3, RMSE: 0.09820821903479503
Site ID: 12, Meter: 0, RMSE: 0.48771092992490117
Site ID: 12, Meter: 0, RMSE: 0.3477324605183933
Site ID: 12, Meter: 0, RMSE: 0.1327676713663535
Site ID: 13, Meter: 0, RMSE: 0.860191373

In [31]:
best_params_df = pd.DataFrame()
for key in models.keys():
    if models[key]:
        try:
            params = models[key].best_params_
            params["best_score"] = models[key].best_score_
        except:
            params = {}
        params["site_id"] = key.split("_")[0]
        params["meter"] = key.split("_")[1]
        params["cluster"] = key.split("_")[2]
        best_params_df = pd.concat([best_params_df, pd.DataFrame(params, index=[key])])

In [32]:
best_params_df.to_csv("BestParamsWithClusterLevelModels.csv")

In [33]:
trials_df = pd.DataFrame()
for key in models.keys():
    if models[key]:
        trial_df = models[key].trials_dataframe()
        trial_df["key"] = key
        trials_df = pd.concat([trials_df, trial_df])

In [34]:
trials_df.to_csv("OptunaStudyTrialsAllWithClusterLevelModels.csv", index=False)